In [1]:
import stumpy
import numpy as np
import random
import math
import pickle
import sys
import os

from statistics import mean
from tqdm.auto import tqdm
from multiprocessing import Pool

import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
'''
Collects random samples from trace with id2 and computes the matrix profile of class1 compared with class 2

Input: 
    trace1: packet traces from class 1
    id2: id number for class 2 
    num_traces: number of traces to select from class 2 (should be equal to class 1)
    shapelet_size: length of shapelets
    
Output:
    Matrix profile of trace1 compared with trace2
'''
def compare_profile(trace1, trace2, shapelet_size):
    
    length_diff = len(trace2) - len(trace1)
    if(length_diff < 0):
        trace2 = np.append(trace2, [np.nan] * abs(length_diff))
        
    #print(len(trace1))
    #print(len(trace2))
        
    
    c1_c2 = stumpy.stump(trace1, shapelet_size, trace2, ignore_trivial=False)[:, 0].astype(float)
    c1_c2[c1_c2 == np.inf] = np.nan
    
    return c1_c2

'''
Compares a the matrix profile of a class trace with itself

Input: 
    trace: packet traces from class 1
    shapelet_size: length of shapelets
    
Output:
    Matrix profile of trace compared with trace
'''

def same_profile(trace, shapelet_size):
    
    c1_c1 = stumpy.stump(trace, shapelet_size)[:, 0].astype(float)
    c1_c1[c1_c1 == np.inf] = np.nan
    
    return c1_c1

'''
return indices of shapelet as one-hot encoded list
'''
def generate_shapelet(trace, diff, shapelet_size):
    
    idx = np.argmax(diff)
    shapelet = np.asarray([1 if idx <= i < idx + shapelet_size else 0 for i in range(len(trace))])
    
    return shapelet

'''
Compute shapelet of greatest overlaps
'''
def find_overlap(trace_i, shapelets_i, shapelet_size):
    #print(shapelets_i[0])
    
    merged_shapelets = np.sum(shapelets_i, axis=0)
    
    max_size = 0
    start = 0
    end = 0
    
    for i in range(0, len(merged_shapelets), shapelet_size):
        current_size = np.sum(merged_shapelets[i:i+shapelet_size])
        if current_size > max_size:
            max_size = current_size
            start = i
            end = i + shapelet_size
    
    return trace_i[start:end]

In [3]:
'''
Generates a set of 100 shapelets for each class in samples

Input:
    num_traces = Number of traces per class
    shapelet_size = Size of shapelets
    save: save results to file?
    filename: if save, name & location of output file

Output:
    list object containing shapelets for each class

'''
def generate_shapelets(shapelet_coeff):
    shapelet_storage = []
    
    # loop over all classes (generate shapelet for each class)
    for i in tqdm(range(100)):
        
        # get the chosen sample from trace i
        trace_i = chosen_traces[i].astype('float64')
        shapelet_size = math.floor(shapelet_coeff * len(trace_i))
        
        shapelets_i = np.zeros((100, len(trace_i)))
        #print(shapelets_i.shape)
        
        # generate profile of i compared with itself
        # length of sample is coeff* len*trace_i
        ci_ci = same_profile(trace_i, shapelet_size)
        
        # loop over every other class and generate a profile for each one
        for j in range(100):
            # don't compare i with itself 
            if i == j:
                continue
            
            trace_j = chosen_traces[j].astype('float64')
            
            # compute profile of i compared with j
            ci_cj = compare_profile(trace_i, trace_j, shapelet_size)

            # find largest value gap between other and i
            diff_ci = ci_cj - ci_ci
            
            # generate best shapelet for i compared to j and store it in list
            ci_shape = generate_shapelet(trace_i, diff_ci, shapelet_size)
            shapelets_i[j] = ci_shape
        
        # compare shapelets between all classes and return the one which has the most overlap
        # (i.e.) the shapelet that was chosen most between the 99 other classes
        best_shapelet = find_overlap(trace_i, shapelets_i, shapelet_size)
        # save to list
        shapelet_storage.append(best_shapelet)
    
    return shapelet_storage   

In [4]:
'''
Compute the minimum distance beteen data samples and shapelets
Input:
    data = list of individual packet traces
    shapelets = list of shapelets
Output:
    minimum distance between each sample in data compared with each sample in shapelet
    shape = (len(data),len(shapelets))
'''
def distance_to_shapelet(data, shapelets):
    #data = np.asarray(data)
    #print(len(data))
    
    # processed output data
    data_out = np.zeros((len(data),len(shapelets)))
    
    # loop over each sample in the dataset
    for i,sample in enumerate(tqdm(data)):
        shapelet_score = np.empty(len(shapelets))
        # for each shapelet, calculate distance and assign a score
        for j,shapelet in enumerate(shapelets):
            try:
                dist = stumpy.mass(shapelet, sample)
            except ValueError:
                dist = stumpy.mass(sample, shapelet)
            shapelet_score[j] = dist.min()
        data_out[i] = shapelet_score
    
    return data_out

'''
Computes distances between input samples and shapelets, returns X for classifier
Also cleans data and ensures no random errors due to length, NaN, etc...
Underlying function that performs comparison is distance_to_shapelet
Selects data samples (with replacement)
note: some samples will always be bad so actual length of X is less

Input:
    num_traces = numner of traces to process
    save = save output to file
    filenames = tuple that represents (name of X file, name of y file)

Output:
    X values for classifier of shape (None, 100)
    y values for classifier of shape (None, )
'''

def process_traces(shapelets):
    X, y = [], []

    
#     for i in range(num_traces):
#         combo_trace = []
#         combo_trace.append(random.choice(traces[random.randint(50,99)]))
#         y_id = random.randint(0,49)
#         combo_trace.append(random.choice(traces[y_id]))
#         combo_trace.append(random.choice(traces[random.randint(50,99)]))
#         out = np.concatenate((combo_trace[0],combo_trace[1],combo_trace[2]))
        
#         X.append(out)
#         y.append(y_id)

    # iterate over dictionary and re-format into X and y
    for trace_id, trace_vals in traces.items():
        for trace in trace_vals:
            X.append(trace)
            y.append(trace_id)
    
    print("Size of X: " + str(len(X)))
    
    
    # convert traces into float64 data type
    X = [np.asarray(trace).astype('float64') for trace in X]
    
    # clear empty trace values in data
    X = [trace[~np.isnan(trace)] for trace in X]    

    # compute distance between input trace and shapelet arrays
    # return as new X

    X = distance_to_shapelet(X, shapelets)
    
    return X, y

In [5]:
'''
Evaluate performance of sklearn classifier on data samples - 90/10 training testing split

Input:
    clf: sklearn classifier object
    X: x values
    y: y values
    topk: k values for evaluation metrics
Output:
    list of length topk with accuracy for testing data
'''

def classifier_performance(clf, X, y, topk=[1,3,5]):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    
    clf.fit(X_train, y_train)
    y_prob = clf.predict_proba(X_test)
    
    scores = []
    for k in topk:
        correct = 0
        for i in range(len(y_prob)):
            ind = np.argpartition(y_prob[i], -k)[-k:]
            if y_test[i] in ind:
                correct += 1
        scores.append(correct/len(y_prob))
    
    return scores

In [6]:
'''
Utility function for pipeline of evaluating different grid search parameters
Output: a new file located in ../results/param1-val1_param2-val2_param3-val3
        the file contains a pickled python object
        with the scores for top-1, top-3, and top-5 classifier accuracy
'''
# note: python multiprocessing is really annoying to work with
# function needs to be in a separate .py file which is imported
# and function can only have 1 argument
# list input which is immediately used for what would be the arguments
def evaluate_parameters(arr):
    
    num_experiment = arr[0]
    shapelet_coeff = arr[1]
    
    filename = '../results/shapelets/' + 'num=' + str(num_experiment) + 'size=' + str(shapelet_coeff)
    with open(filename, 'rb') as f:
        shapelets = pickle.load(f)
    
    shapelets = [shapelet.astype('float64') for shapelet in shapelets]
    
    X, y = process_traces(shapelets)
    
    filename = '../results/data/X/' + 'num=' + str(num_experiment) + 'size=' + str(shapelet_coeff)
    
    with open(filename, 'wb') as f:
        pickle.dump(X, f)
        
    filename = '../results/data/y/' + 'num=' + str(num_experiment) + 'size=' + str(shapelet_coeff)
    
    with open(filename, 'wb') as f:
        pickle.dump(y, f)

In [7]:
# SETUP

global traces

with open('../ds19.npy', 'rb') as f:
    traces = pickle.load(f)

In [9]:
nums = range(36)
size = [0]
parameter_list = [[x,y] for x in nums for y in size]

print(parameter_list)

[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [7, 0], [8, 0], [9, 0], [10, 0], [11, 0], [12, 0], [13, 0], [14, 0], [15, 0], [16, 0], [17, 0], [18, 0], [19, 0], [20, 0], [21, 0], [22, 0], [23, 0], [24, 0], [25, 0], [26, 0], [27, 0], [28, 0], [29, 0], [30, 0], [31, 0], [32, 0], [33, 0], [34, 0], [35, 0]]


In [ ]:
# PART 1

print(parameter_list)

for parameters in parameter_list:
    coeff = parameters[1]
    shapelets = generate_shapelets(coeff)
    
    filename = '../results/shapelets/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1])
    
    with open(filename, 'wb') as f:
        pickle.dump(shapelets, f)   

In [10]:
## PART 2

if __name__ == '__main__':

    from utils import evaluate_parameters
    print(parameter_list)
    
    with Pool(6) as p:
        p.map(evaluate_parameters, parameter_list)

[[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [7, 0], [8, 0], [9, 0], [10, 0], [11, 0], [12, 0], [13, 0], [14, 0], [15, 0], [16, 0], [17, 0], [18, 0], [19, 0], [20, 0], [21, 0], [22, 0], [23, 0], [24, 0], [25, 0], [26, 0], [27, 0], [28, 0], [29, 0], [30, 0], [31, 0], [32, 0], [33, 0], [34, 0], [35, 0]]
Size of X: 10000
Size of X: 10000
Size of X: 10000
Size of X: 10000
Size of X: 10000
Size of X: 10000


 94%|█████████▍| 9398/10000 [55:10<08:38,  1.16it/s]

Size of X: 10000


 97%|█████████▋| 9734/10000 [56:19<00:31,  8.49it/s]

Size of X: 10000


...

 12%|█▏        | 1216/10000 [04:49<35:53,  4.08it/s]

Size of X: 10000


  0%|          | 1/10000 [00:00<31:26,  5.30it/s]/s]

Size of X: 10000


100%|██████████| 10000/10000 [58:31<00:00,  2.85it/s]


In [13]:
# merge X values from different datasets

folder_X = "../results/data/X/"
names = ["num=0size=1","num=1size=1","num=2size=1","num=3size=1","num=4size=1","num=5size=1"]

X = ()

for filename in names: #os.listdir(folder_X): (for all files)
    print(filename)
    with open(folder_X + filename, 'rb') as f:
        Xi = pickle.load(f)
    X = X + (Xi,) 
        
X = np.concatenate(X, axis=1)
print(X.shape)

with open("../results/data/y/num=0size=1", 'rb') as f:
    y = pickle.load(f)
y = np.array(y) 

print(y.shape)

num=0size=1
num=1size=1
num=2size=1
num=3size=1
num=4size=1
num=5size=1
(10000, 600)
(10000,)


In [24]:
clf = RandomForestClassifier()
scores = classifier_performance(clf, X, y)
    
print(scores)

[0.892, 0.959, 0.973]


In [10]:
## PART 3

print(parameter_list)

for parameters in parameter_list:
    
    filename = '../results/data/X/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1])
    
    with open(filename, 'rb') as f:
        X = pickle.load(f)
    
    filename = '../results/data/y/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1])
    
    with open(filename, 'rb') as f:
        y = pickle.load(f)
    
    clf = RandomForestClassifier()
    scores = classifier_performance(clf, X, y)
    
    print(scores)
    
    outfile_name = "../results/scores/" + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1])
    
    with open(outfile_name, 'wb') as f:
        pickle.dump(scores, f)

[['0', '0']]
[0.875, 0.944, 0.965]


In [ ]:
# PART 4

print(parameter_list)

for parameters in parameter_list:
    
    filename = '../results/data/X/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1])
    
    with open(filename, 'rb') as f:
        X = pickle.load(f)
        
    filename = '../results/data/y/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1])
    
    with open(filename, 'rb') as f:
        y = pickle.load(f)
    
    clf = RandomForestClassifier()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    matrix = confusion_matrix(y_test, y_pred)
    scores = matrix.diagonal()/matrix.sum(axis=1)
    
    outfile_name = "../results/scores_perclass/" + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1])
    
    with open(outfile_name, 'wb') as f:
        pickle.dump(scores, f)

In [27]:
import xgboost as xgb

# Create regression matrices
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)


params = {
    "objective": "multi:softprob",
    "num_class": 100,
    "tree_method": "exact",
    "device": "cpu",
}

model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=150,
    verbose_eval = True
)

preds = model.predict(dtest_reg)

In [28]:
scores = []
topk=[1,3,5]

for k in topk:
    correct = 0
    for i in range(len(preds)):
        ind = np.argpartition(preds[i], -k)[-k:]
        if y_test[i] in ind:
            correct += 1
    scores.append(correct/len(preds))
    
print(scores)

[0.855, 0.948, 0.966]


In [29]:
import tensorflow as tf
import tensorflow.keras as keras

from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy
from keras.layers import Dense, Dropout

model = Sequential([
    Dense(1024),
    Dense(512),
    Dense(256),
    Dense(128),
    Dense(100)
])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

2024-01-24 14:06:55.169391: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 14:07:05.389910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
282/282 [==============================] - 1s 3ms/step - loss: 317.3727 - accuracy: 0.0292
Epoch 2/1000
282/282 [==============================] - 1s 3ms/step - loss: 73.9280 - accuracy: 0.0646
Epoch 3/1000
282/282 [==============================] - 1s 3ms/step - loss: 34.7955 - accuracy: 0.1162
Epoch 4/1000
282/282 [==============================] - 1s 3ms/step - loss: 16.8211 - accuracy: 0.1792
Epoch 5/1000
282/282 [==============================] - 1s 3ms/step - loss: 11.8718 - accuracy: 0.2368
Epoch 6/1000
282/282 [==============================] - 1s 3ms/step - loss: 9.0857 - accuracy: 0.2670
Epoch 7/1000
282/282 [==============================] - 1s 3ms/step - loss: 7.8939 - accuracy: 0.2911
Epoch 8/1000
282/282 [==============================] - 1s 3ms/step - loss: 7.9087 - accuracy: 0.2829
Epoch 9/1000
282/282 [==============================] - 1s 3ms/step - loss: 7.0620 - accuracy: 0.3092
Epoch 10/1000
282/282 [==============================] - 1s 3ms/step - loss:

Epoch 159/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.5564 - accuracy: 0.6642
Epoch 160/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.9566 - accuracy: 0.6160
Epoch 161/1000
282/282 [==============================] - 1s 3ms/step - loss: 2.3499 - accuracy: 0.5637
Epoch 162/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.5704 - accuracy: 0.6389
Epoch 163/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.4235 - accuracy: 0.6531
Epoch 164/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.8409 - accuracy: 0.5832
Epoch 165/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.4000 - accuracy: 0.6429
Epoch 166/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.5469 - accuracy: 0.6220
Epoch 167/1000
282/282 [==============================] - 1s 3ms/step - loss: 2.0087 - accuracy: 0.5438
Epoch 168/1000
282/282 [==============================] - 1s 3ms

282/282 [==============================] - 1s 3ms/step - loss: 1.3453 - accuracy: 0.7236
Epoch 316/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.9477 - accuracy: 0.7798
Epoch 317/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7720 - accuracy: 0.8054
Epoch 318/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7222 - accuracy: 0.8117
Epoch 319/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.7764 - accuracy: 0.8023
Epoch 320/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.6939 - accuracy: 0.8213
Epoch 321/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.6952 - accuracy: 0.8181
Epoch 322/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.6715 - accuracy: 0.8233
Epoch 323/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7411 - accuracy: 0.8029
Epoch 324/1000
282/282 [==============================] - 1s 3ms/step - loss: 0

282/282 [==============================] - 1s 3ms/step - loss: 1.0640 - accuracy: 0.7501
Epoch 472/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.0447 - accuracy: 0.7514
Epoch 473/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.1080 - accuracy: 0.7313
Epoch 474/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.0318 - accuracy: 0.7354
Epoch 475/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.0510 - accuracy: 0.7281
Epoch 476/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.9001 - accuracy: 0.7589
Epoch 477/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.0467 - accuracy: 0.7246
Epoch 478/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.2534 - accuracy: 0.6818
Epoch 479/1000
282/282 [==============================] - 1s 3ms/step - loss: 1.2926 - accuracy: 0.6721
Epoch 480/1000
282/282 [==============================] - 1s 3ms/step - loss: 1

282/282 [==============================] - 1s 3ms/step - loss: 0.6413 - accuracy: 0.8301
Epoch 628/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.7161 - accuracy: 0.8127
Epoch 629/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.6531 - accuracy: 0.8268
Epoch 630/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.8273 - accuracy: 0.7823
Epoch 631/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.7079 - accuracy: 0.8066
Epoch 632/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.7274 - accuracy: 0.8029
Epoch 633/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.9945 - accuracy: 0.7450
Epoch 634/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.9433 - accuracy: 0.7556
Epoch 635/1000
282/282 [==============================] - 1s 4ms/step - loss: 1.0173 - accuracy: 0.7323
Epoch 636/1000
282/282 [==============================] - 1s 3ms/step - loss: 0

282/282 [==============================] - 1s 4ms/step - loss: 0.5696 - accuracy: 0.8498
Epoch 784/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.6047 - accuracy: 0.8476
Epoch 785/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.6236 - accuracy: 0.8420
Epoch 786/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7483 - accuracy: 0.8194
Epoch 787/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.8365 - accuracy: 0.8046
Epoch 788/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.7722 - accuracy: 0.8068
Epoch 789/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.8070 - accuracy: 0.8064
Epoch 790/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7368 - accuracy: 0.8052
Epoch 791/1000
282/282 [==============================] - 1s 4ms/step - loss: 0.7150 - accuracy: 0.8081
Epoch 792/1000
282/282 [==============================] - 1s 3ms/step - loss: 0

282/282 [==============================] - 1s 3ms/step - loss: 0.9064 - accuracy: 0.8361
Epoch 940/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.8090 - accuracy: 0.8497
Epoch 941/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7798 - accuracy: 0.8442
Epoch 942/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7909 - accuracy: 0.8463
Epoch 943/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7464 - accuracy: 0.8489
Epoch 944/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.6963 - accuracy: 0.8540
Epoch 945/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7229 - accuracy: 0.8510
Epoch 946/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.7421 - accuracy: 0.8402
Epoch 947/1000
282/282 [==============================] - 1s 3ms/step - loss: 0.6240 - accuracy: 0.8563
Epoch 948/1000
282/282 [==============================] - 1s 3ms/step - loss: 0

In [32]:
predictions = model.predict(X_test)
predictions = tf.nn.softmax(predictions)

score = 0
for i, pred in enumerate(predictions):
    final_pred = np.argmax(pred, 0)
    
    if final_pred == y_test[i]:
        score += 1

print(score/len(y_test))

32/32 [==============================] - 0s 1ms/step
0.583
